In [1]:
import os
import time
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
'''ADD MORE IF NEEDED'''
from torch.nn import Module, Linear, ReLU, Conv2d, Sequential , CrossEntropyLoss
from  torch.utils.data import DataLoader, random_split
from torch.optim import Adam,lr_scheduler
from torchvision.transforms import Compose, ToTensor, Normalize 
''''''


''

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [19]:
'''
    ADD THE PATH TO VIEW IN TENSORBOARD
'''
PATH = " "
writer = SummaryWriter('runs/' + PATH)

In [12]:


class ClassModel(Module):
    def __init__(self):
        '''
            CHANGE TO APPLY (here 28*28 use for example MNIST)
        '''
        super(ClassModel, self).__init__()
        self.L1 = Linear(in_features=28*28, out_features=256) 
        self.L2 = Linear(in_features=256, out_features=128)
        self.L3 = Linear(in_features=128, out_features=64)
        self.L4 = Linear(in_features=64, out_features=10)
        self.relu = ReLU()

        '''
            NOT CHANGE HERE
        '''
        self.train_dataset = None 
        self.val_dataset = None 
        self.test_dataset = None
        self.optimizer= None 
        self.step_lr_scheduler = None
        self.losses = None
        self.checkpoint = None
        self.flag_tensorboard = False
        self.flag_step = False

    def forward(self, x):
        '''
        CHANGE TO APPLY
        '''
        x = self.L1(x)
        x = self.relu(x)
        x = self.L2(x)
        x = self.relu(x)
        x = self.L3(x)
        x = self.relu(x)
        x = self.L4(x)
        return x
    
    def prepare_data(self, percentage_val=0.2):
        '''
            CHANGE TO APPLY
        '''
        name_dataset = torchvision.datasets.MNIST
        train_dataset_full = name_dataset(
            train=True,
            transform=Compose([ToTensor()]),
            root='./data/',
            download=True,
        )
        self.test_dataset = name_dataset(
            train=False,
            transform=Compose([ToTensor()]),
            root='./data/',
            download=False,
        )
        train_size = int((1.0-percentage_val)*len(train_dataset_full))
        val_size = int(percentage_val*len(train_dataset_full))
        self.train_dataset, self.val_dataset = random_split(train_dataset_full, lengths=[train_size,val_size])      
        return self.train_dataset, self.val_dataset, self.test_dataset
    
    def train_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        self.train_dataset = DataLoader(
            dataset=self.train_dataset,
            batch_size=batch_size,
            shuffle=True
        )
        return self.train_dataset
        
    def val_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        self.val_dataset = DataLoader(
            dataset=self.val_dataset,
            batch_size=batch_size,
            shuffle=False
        )
        return self.val_dataset

    def test_dataloader(self, batch_size=64):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        self.test_dataset = DataLoader(
            dataset=self.test_dataset,
            batch_size=batch_size,
            shuffle=False
        )
        return self.test_dataset
    
    def configure_optimizer(self):
        '''
            CHANGE LEARNING_RATE, LOSS_ENTROPY,CLASS LR_SCHEDULER(step_size == epoch) TO APPLY
        '''
        learning_rate = 0.001
        self.optimizer = Adam(self.parameters(), lr=learning_rate)
        self.losses = CrossEntropyLoss()
        self.step_lr_scheduler = lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)
        return self.optimizer, self.step_lr_scheduler, self.losses
    
    def training_step(self, batch, batch_idx):
        data, target = batch
        data = data.reshape(-1,28*28).to(device)
        target = target.to(device)
        output = self(data)
        loss = self.losses(output, target)
        _, predictions = torch.max(output, 1)
        accuracy = torch.sum(predictions == target).item()

        #backward
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.item(), accuracy
    
    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            data, target = batch
            data = data.reshape(-1, 28*28).to(device)
            target = target.to(device)
            output = self(data)
            loss = self.losses(output, target)
            _, predictions = torch.max(output, 1)
            accuracy = torch.sum(predictions == target).item()

            return loss.item(), accuracy
        
    def validation_epoch_end(self, outputs_loss, outputs_accuracy, n_samples):
        avg_loss = outputs_loss / n_samples
        avg_accuracy = outputs_accuracy / n_samples
        return avg_loss, avg_accuracy

    
    def train_model(self, epochs, intervals=100):
        '''
            CHANGE TO EPOCH TO APPLY
        '''
        for epoch in range(epochs):
            print("Epoch {}/{}".format(epoch + 1, epochs))
            since = time.time()
            epoch_train_loss = 0
            epoch_train_accuracy = 0
            epoch_val_loss = 0
            epoch_val_accuracy = 0
            for phase in ['train','val']:
                run_loss = 0.0
                run_accuracy = 0.0
                n_samples = 0

                if phase == 'train':
                    self.train()
                    for i, (data, target) in enumerate(self.train_dataset):
                        n_samples += len(target)
                        loss, accuracy = self.training_step((data, target), i)
                        run_loss += loss * data.size(0)
                        run_accuracy += accuracy
                    
                        if i % intervals == 0 and self.flag_step == True:
                            print(f"Training Process - Step {i} / {len(self.train_dataset)}, loss = {loss:.4f}")
                            if self.flag_tensorboard == True:
                                writer.add_scalar("training loss",scalar_value=(run_loss / intervals ), global_step=(epoch * len(self.train_dataset) + i))
                                writer.add_scalar("training accuracy",scalar_value=(run_accuracy / intervals ), global_step=(epoch * len(self.train_dataset) + i))
                    epoch_train_loss = run_loss / n_samples
                    epoch_train_accuracy = run_accuracy / n_samples
                    self.step_lr_scheduler.step()
                
                
                else:
                    self.eval()
                    for i, (data, target) in enumerate(self.val_dataset):
                        n_samples += len(target)
                        loss, accuracy = self.validation_step((data, target), i)
                        run_loss += loss * data.size(0)
                        run_accuracy += accuracy

                        if i % intervals == 0 and self.flag_step == True:
                            print(f"Validation Process - Step {i} / {len(self.val_dataset)}, loss = {loss:.4f}")
                            if self.flag_tensorboard == True:
                                writer.add_scalar("validation loss",scalar_value=(run_loss / intervals ), global_step=(epoch * len(self.val_dataset) + i))
                                writer.add_scalar("validation accuracy",scalar_value=(run_accuracy / intervals ), global_step=(epoch * len(self.val_dataset) + i))
                    epoch_val_loss, epoch_val_accuracy = self.validation_epoch_end(run_loss, run_accuracy, n_samples)

            time_elapsed = time.time() - since
            print(f"{time_elapsed:.4f}s/step - loss: {epoch_train_loss:.4f} - accuracy: {epoch_train_accuracy:.4f} - val_loss: {epoch_val_loss:.4f} - val_accuracy: {epoch_val_accuracy:.4f}")
            print("-"*30)
            self.save_model('model.pth')  

    def test_model(self):
        self.eval()
        n_samples = 0
        run_accuracy = 0
        for i, (data, target) in enumerate(self.test_dataset):
            n_samples += len(target)
            _, accuracy = self.validation_step((data, target), i)
            run_accuracy += accuracy
        
        accuracy = 100 * run_accuracy / n_samples
        print(f"Accuracy of test dataset: {accuracy}%")

    def save_model(self, PATH):
        return torch.save(self.state_dict(), PATH)
    
    def load_state_dict(self, PATH):
        return self.load_state_dict(torch.load(PATH))

    def take_checkpoint(self, epoch, loss, PATH):
        self.checkpoint ={
            'epoch': epoch,
            'model_state_dict': self.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': loss
        }
        return torch.save(self.checkpoint, PATH)
    
    def load_checkpoint(self, PATH):
        self.checkpoint = torch.load(PATH)
        self.load_state_dict(self.checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(self.checkpoint['optimizer_state_dict'])
        epoch = self.checkpoint['epoch']
        loss = self.checkpoint['loss']
        return epoch, loss

In [13]:
model = ClassModel()

In [14]:
model.prepare_data()
model.train_dataloader()
model.test_dataloader()
model.val_dataloader()
# model.flag_tensorboard = True
# model.flag_step = True

In [15]:
print(len(model.train_dataset)*64)
print(len(model.val_dataset)*64)
print(len(model.test_dataset)*64)

48000
12032
10048


In [16]:
model.configure_optimizer()

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     initial_lr: 0.001
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [17]:
model.train_model(epochs=5)

Epoch 1/5
3.7586s/step - loss: 0.3465 - accuracy: 0.8935 - val_loss: 0.1934 - val_accuracy: 0.9415
------------------------------
Epoch 2/5
3.6545s/step - loss: 0.1282 - accuracy: 0.9614 - val_loss: 0.1161 - val_accuracy: 0.9656
------------------------------
Epoch 3/5
3.5218s/step - loss: 0.0848 - accuracy: 0.9740 - val_loss: 0.1040 - val_accuracy: 0.9693
------------------------------
Epoch 4/5
3.6461s/step - loss: 0.0621 - accuracy: 0.9804 - val_loss: 0.0909 - val_accuracy: 0.9732
------------------------------
Epoch 5/5
3.7244s/step - loss: 0.0476 - accuracy: 0.9848 - val_loss: 0.1078 - val_accuracy: 0.9676
------------------------------


In [18]:
model.test_model()


Accuracy of test dataset: 97.13%
